In [ ]:
# 유클리드 거리
import numpy as np

def dist(x,y):  
    return np.sqrt(np.sum((x-y)**2))

doc0 = np.array((1,1,0,1))
doc1 = np.array((2,3,0,1))
doc2 = np.array((1,2,3,1))

print(dist(doc0,doc1)) #doc0과 doc1의 거리
print(dist(doc0,doc2))

In [ ]:
# 자카드 유사도: 두 문서의 총 단어 집합에서 공통적으로 출현한 단어의 비율

doc1 = "python 파이썬 데이터"
doc2 = "빅데이터 python 파이썬"

# 토큰화
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()

print(tokenized_doc1)
print(tokenized_doc2)

In [ ]:
# 합집합

union = set(tokenized_doc1).union(set(tokenized_doc2))

print(union)

In [ ]:
# 교집합

intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))

print(intersection)

In [ ]:
print(len(intersection)/len(union))

In [ ]:
# Cosine Similarity
# 두 벡터 간의 코사인 각도를 이용하여 구하는 두 벡터의 유사도
# 두 벡터의 방향이 완전히 같으면 1, 90도이면 0, 반대 방향이면 -1
# 1에 가까울수록 유사도가 높다고 판단함
from numpy import dot

a=[0,1,1]
b=[1,0,2]

# 배열의 곱( 0x1 + 1x0 + 1x2)
dot(a,b)

In [ ]:
from math import sqrt
from numpy.linalg import norm

a=[0,1,1]
b=[1,0,2]

print(norm(a)) #a의 제곱합의 제곱근
print(sqrt(2))
print(norm(b))
print(sqrt(5))
print(norm(a)*norm(b))

In [ ]:
# a,b의 코사인 유사도

print( dot(a,b) / (norm(a)*norm(b)))

In [ ]:
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

In [ ]:
import numpy as np

doc1=np.array([0,1,1,1])
doc2=np.array([1,0,1,1])
doc3=np.array([2,0,2,3])
doc4=np.array([0,2,2,2])

print(cos_sim(doc1, doc2)) # 문서1과 문서2의 코사인 유사도
print(cos_sim(doc1, doc3)) # 문서1과 문서3의 코사인 유사도
print(cos_sim(doc2, doc3)) # 문서2과 문서3의 코사인 유사도
print(cos_sim(doc1, doc4)) # 문서1과 문서4의 코사인 유사도

# 코사인 유사도는 단순한 빈도수보다도 두 벡터의 방향이 완전히 동일한 경우에는 1(유사도가 최대)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
 '매우 좋은 영화네요 매우 추천해요',
 '매우 볼만한 영화네요.',
 '조금 볼만한 영화네요 조금 추천해요',
 '별로 볼 내용이 없는 것 같아요 추천하지 않아요',
]

# DTM(Document Term Matrix, 문서 단어 행렬)

vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수 계산
print(vector.fit_transform(corpus).toarray())
print(vector.vocabulary_)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer  

tfidfv = TfidfVectorizer().fit(corpus)  
tfidf_matrix = tfidfv.fit_transform(corpus)  

print(tfidfv)  
print(tfidfv.transform(corpus).toarray())  
print(tfidfv.vocabulary_)

In [ ]:
from sklearn.metrics.pairwise import linear_kernel  

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)  
cosine_sim

In [ ]:
#코사인 유사도를 계산하여 줄거리가 유사한 영화 추천
import pandas as pd

#https://www.kaggle.com/rounakbanik/the-movies-dataset

df = pd.read_csv('c:/data/movies/movies_metadata.csv',low_memory=False)
df.head()

# 일부 컬럼에 자료형이 혼합된 경우 메모리 사용량이 증가할 수 있으므로 low_memory=False 사용

In [ ]:
df=df.head(10000) #1만개의 행으로 실습

# overview(줄거리) 필드의 결측값이 있는 행의 수

df['overview'].isnull().sum()

In [ ]:
# 결측값을 빈값으로 채움

df['overview'] = df['overview'].fillna('')
df['overview'].isnull().sum()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

# overview에 대해서 tf-idf 수행

tfidf_matrix = tfidf.fit_transform(df['overview'])
print(tfidf_matrix.shape)

# 단어 개수 32350

In [ ]:
for idx,value in enumerate(tfidf_matrix[0].toarray()[0]):  
    if value>0:
        print(idx, value)

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

# tfidf에서는 dot product를 구하면 코사인 유사도를 얻을 수 있음

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

linear_kernel([[1,2]],[[1,2]]) # 1x1 + 2x2

In [ ]:
# 영화 제목과 인덱스, drop_duplicates() 중복값 제거

indices = pd.Series(df.index, index=df['title']).drop_duplicates()

print(indices.head())

In [ ]:
# 영화제목을 입력하면 인덱스가 리턴됨

idx = indices['Toy Story']
idx

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):

    # 영화의 제목으로 인덱스 조회
    idx = indices[title]

    # 해당 영화와의 유사도 계산
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 정렬, key 정렬기준 필드(두번째값 기준 정렬), reverse 내림차순
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화 리스트
    sim_scores = sim_scores[1:11]
    print(sim_scores)

    # 리스트의 0번 인덱스
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목
    return df['title'].iloc[movie_indices]

In [ ]:
# 코사인 유사도를 기준으로 유사한 영화 목록 추천

get_recommendations('Toy Story')

In [ ]:
get_recommendations('The Show')